In [98]:
import pandas as pd
import numpy as np
import json
from io import StringIO
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from math import ceil
import datetime
from datetime import datetime 
import calendar
from sqlalchemy import create_engine
import psycopg2

In [99]:
# Specify the path to the JSON configuration file
config_file_path = "/Users/gabisanches/Desktop/CIS9440 - Data Warehouse/Homework/Homework_GabrieleSanches_CIS9440/scripts/config.json"

# Load the JSON configuration file
with open(config_file_path, 'r') as config_file:
    config = json.load(config_file)

CONNECTION_STRING_AZURE_STORAGE = config["connectionString"]
CONTAINER_AZURE = 'carcrash'

# Initialize the BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING_AZURE_STORAGE)
# Get the container client
container_client = blob_service_client.get_container_client(CONTAINER_AZURE)

car_crash_df = pd.DataFrame()

# List all blobs in the specified container
blob_list = container_client.list_blobs()
for blob in blob_list:
    print(blob.name)
    blob_client = container_client.get_blob_client(blob=blob.name)
    blob_data = blob_client.download_blob()
    blob_content = blob_data.readall().decode('utf-8')
    df = pd.read_csv(StringIO(blob_content))
    # Show the head of the DataFrame
    print(df.shape)
    car_crash_df = df.copy()

car_crash.csv


/var/folders/y5/y4jkcjwx1zsdr2dqg2zg998r0000gn/T/ipykernel_6777/991417440.py:27: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(StringIO(blob_content))


(2077866, 29)


In [100]:
# Print columns in the data set
print(car_crash_df.columns)

Index(['crash_date', 'crash_time', 'on_street_name', 'off_street_name',
       'number_of_persons_injured', 'number_of_persons_killed',
       'number_of_pedestrians_injured', 'number_of_pedestrians_killed',
       'number_of_cyclist_injured', 'number_of_cyclist_killed',
       'number_of_motorist_injured', 'number_of_motorist_killed',
       'contributing_factor_vehicle_1', 'contributing_factor_vehicle_2',
       'collision_id', 'vehicle_type_code1', 'vehicle_type_code2', 'borough',
       'zip_code', 'latitude', 'longitude', 'location', 'cross_street_name',
       'contributing_factor_vehicle_3', 'vehicle_type_code_3',
       'contributing_factor_vehicle_4', 'vehicle_type_code_4',
       'contributing_factor_vehicle_5', 'vehicle_type_code_5'],
      dtype='object')


In [101]:
# Drop columns that are not needed
car_crash_df = car_crash_df.drop(columns=['contributing_factor_vehicle_3'])
car_crash_df = car_crash_df.drop(columns=['vehicle_type_code_3'])
car_crash_df = car_crash_df.drop(columns=['contributing_factor_vehicle_4'])
car_crash_df = car_crash_df.drop(columns=['vehicle_type_code_4'])
car_crash_df = car_crash_df.drop(columns=['contributing_factor_vehicle_5'])
car_crash_df = car_crash_df.drop(columns=['vehicle_type_code_5'])
car_crash_df = car_crash_df.drop(columns=['contributing_factor_vehicle_2'])
car_crash_df = car_crash_df.drop(columns=['vehicle_type_code2'])
car_crash_df = car_crash_df.drop(columns=['location'])
car_crash_df = car_crash_df.drop(columns=['cross_street_name'])
car_crash_df = car_crash_df.drop(columns=['off_street_name'])

In [103]:
# Reformt date column
car_crash_df['crash_date'] = pd.to_datetime(car_crash_df['crash_date'])

# Extract only the date portion
car_crash_df['crash_date'] = car_crash_df['crash_date'].dt.date

In [104]:
car_crash_df.head()

,crash_date,crash_time,on_street_name,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,contributing_factor_vehicle_1,collision_id,vehicle_type_code1,borough,zip_code,latitude,longitude
0,2021-09-11,2:39,WHITESTONE EXPRESSWAY,2.0,0.0,0,0,0,0,2,0,Aggressive Driving/Road Rage,4455765,Sedan,NaN,NaN,NaN,NaN
1,2022-03-26,11:45,QUEENSBORO BRIDGE UPPER,1.0,0.0,0,0,0,0,1,0,Pavement Slippery,4513547,Sedan,NaN,NaN,NaN,NaN
2,2022-06-29,6:55,THROGS NECK BRIDGE,0.0,0.0,0,0,0,0,0,0,Following Too Closely,4541903,Sedan,NaN,NaN,NaN,NaN
3,2021-09-11,9:35,NaN,0.0,0.0,0,0,0,0,0,0,Unspecified,4456314,Sedan,BROOKLYN,11208.0,40.667202,-73.866500
4,2021-12-14,8:13,SARATOGA AVENUE,0.0,0.0,0,0,0,0,0,0,NaN,4486609,NaN,BROOKLYN,11233.0,40.683304,-73.917274


In [105]:
# Rename columns
car_crash_df.rename(columns={'contributing_factor_vehicle_1': 'contributing_factor_vehicle'}, inplace=True)
car_crash_df.rename(columns={'vehicle_type_code1': 'vehicle_type_code'}, inplace=True)

In [106]:
# Check column names after initial cleaning and transformation
print(car_crash_df.columns)

Index(['crash_date', 'crash_time', 'on_street_name',
       'number_of_persons_injured', 'number_of_persons_killed',
       'number_of_pedestrians_injured', 'number_of_pedestrians_killed',
       'number_of_cyclist_injured', 'number_of_cyclist_killed',
       'number_of_motorist_injured', 'number_of_motorist_killed',
       'contributing_factor_vehicle', 'collision_id', 'vehicle_type_code',
       'borough', 'zip_code', 'latitude', 'longitude'],
      dtype='object')


In [107]:
# Drop rows
car_crash_df = car_crash_df.dropna(subset=['on_street_name'])
car_crash_df.dropna(subset=['longitude', 'latitude'], inplace=True)
car_crash_df.dropna(subset=['borough', 'zip_code'], inplace=True)
car_crash_df.head()

,crash_date,crash_time,on_street_name,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,contributing_factor_vehicle,collision_id,vehicle_type_code,borough,zip_code,latitude,longitude
4,2021-12-14,8:13,SARATOGA AVENUE,0.0,0.0,0,0,0,0,0,0,NaN,4486609,NaN,BROOKLYN,11233.0,40.683304,-73.917274
9,2021-12-14,14:58,3 AVENUE,0.0,0.0,0,0,0,0,0,0,Passing Too Closely,4486519,Sedan,MANHATTAN,10017.0,40.751440,-73.973970
11,2021-12-14,16:50,SPRINGFIELD BOULEVARD,0.0,0.0,0,0,0,0,0,0,Turning Improperly,4487127,Sedan,QUEENS,11413.0,40.675884,-73.755770
14,2021-12-14,23:10,NORTH CONDUIT AVENUE,2.0,0.0,0,0,0,0,2,0,Reaction to Uninvolved Vehicle,4486635,Sedan,QUEENS,11434.0,40.666840,-73.789410
18,2021-12-11,19:43,WEST KINGSBRIDGE ROAD,1.0,0.0,0,0,0,0,1,0,Unspecified,4487040,Station Wagon/Sport Utility Vehicle,BRONX,10463.0,40.872620,-73.904686


In [108]:
# Reformat data types to int (format) 
car_crash_df['number_of_persons_injured'] = car_crash_df['number_of_persons_injured'].astype('Int64')
car_crash_df['number_of_persons_killed'] = car_crash_df['number_of_persons_killed'].astype('Int64')
car_crash_df['number_of_pedestrians_injured'] = car_crash_df['number_of_pedestrians_injured'].astype('Int64')
car_crash_df['number_of_pedestrians_killed'] = car_crash_df['number_of_pedestrians_killed'].astype('Int64')
car_crash_df['number_of_cyclist_injured'] = car_crash_df['number_of_cyclist_injured'].astype('Int64')
car_crash_df['number_of_cyclist_killed'] = car_crash_df['number_of_cyclist_killed'].astype('Int64')
car_crash_df['number_of_motorist_injured'] = car_crash_df['number_of_motorist_injured'].astype('Int64')
car_crash_df['number_of_motorist_killed'] = car_crash_df['number_of_motorist_killed'].astype('Int64')

car_crash_df.head()

,crash_date,crash_time,on_street_name,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,contributing_factor_vehicle,collision_id,vehicle_type_code,borough,zip_code,latitude,longitude
4,2021-12-14,8:13,SARATOGA AVENUE,0,0,0,0,0,0,0,0,NaN,4486609,NaN,BROOKLYN,11233.0,40.683304,-73.917274
9,2021-12-14,14:58,3 AVENUE,0,0,0,0,0,0,0,0,Passing Too Closely,4486519,Sedan,MANHATTAN,10017.0,40.751440,-73.973970
11,2021-12-14,16:50,SPRINGFIELD BOULEVARD,0,0,0,0,0,0,0,0,Turning Improperly,4487127,Sedan,QUEENS,11413.0,40.675884,-73.755770
14,2021-12-14,23:10,NORTH CONDUIT AVENUE,2,0,0,0,0,0,2,0,Reaction to Uninvolved Vehicle,4486635,Sedan,QUEENS,11434.0,40.666840,-73.789410
18,2021-12-11,19:43,WEST KINGSBRIDGE ROAD,1,0,0,0,0,0,1,0,Unspecified,4487040,Station Wagon/Sport Utility Vehicle,BRONX,10463.0,40.872620,-73.904686


In [109]:
print(car_crash_df.columns)

Index(['crash_date', 'crash_time', 'on_street_name',
       'number_of_persons_injured', 'number_of_persons_killed',
       'number_of_pedestrians_injured', 'number_of_pedestrians_killed',
       'number_of_cyclist_injured', 'number_of_cyclist_killed',
       'number_of_motorist_injured', 'number_of_motorist_killed',
       'contributing_factor_vehicle', 'collision_id', 'vehicle_type_code',
       'borough', 'zip_code', 'latitude', 'longitude'],
      dtype='object')


In [110]:
print(car_crash_df.shape)

(1086268, 18)


In [111]:
#Create vehicle_type dimension

# Get unique values from the 'vehicle_type_code' column
unique_vehicle_types = car_crash_df['vehicle_type_code'].unique()

# Create DataFrame for the dimension table
dim_vehicle_type = pd.DataFrame(unique_vehicle_types, columns=['vehicle_type_code'])

# Add primary key column
dim_vehicle_type['vehicle_id'] = range(1, len(dim_vehicle_type) + 1)

# Rearrange columns if needed
dim_vehicle_type = dim_vehicle_type[['vehicle_id', 'vehicle_type_code']]

# Print dimension table
print(dim_vehicle_type)


     vehicle_id                    vehicle_type_code
0             1                                  NaN
1             2                                Sedan
2             3  Station Wagon/Sport Utility Vehicle
3             4                                  Bus
4             5                                 Taxi
..          ...                                  ...
835         836                            Econoline
836         837                           Lime Scoot
837         838                           moped scoo
838         839                            USPS Mail
839         840                           charter bu

[840 rows x 2 columns]


In [112]:
#Create contributing_factors dimension

# Get unique values from the 'contributing_factor_vehicle' column
unique_contributing_factors = car_crash_df['contributing_factor_vehicle'].dropna().unique()

# Create DataFrame for the dimension table
dim_contributing_factors = pd.DataFrame(unique_contributing_factors, columns=['contributing_factor'])

# Add primary key column
dim_contributing_factors['contributingfactor_id'] = range(1, len(dim_contributing_factors) + 1)

# Rearrange columns if needed
dim_contributing_factors = dim_contributing_factors[['contributingfactor_id', 'contributing_factor']]

# Print dimension table
print(dim_contributing_factors)

    contributingfactor_id                                contributing_factor
0                       1                                Passing Too Closely
1                       2                                 Turning Improperly
2                       3                     Reaction to Uninvolved Vehicle
3                       4                                        Unspecified
4                       5                     Passing or Lane Usage Improper
5                       6                                Alcohol Involvement
6                       7                     Driver Inattention/Distraction
7                       8                            View Obstructed/Limited
8                       9                        Traffic Control Disregarded
9                      10                      Failure to Yield Right-of-Way
10                     11                              Following Too Closely
11                     12                                 Lost Consciousness

In [113]:
#Create location dimension

# Extract relevant columns for dim_location
dim_location = car_crash_df[['borough', 'latitude', 'longitude', 'zip_code', 'on_street_name']]

# Create a new DataFrame after dropping duplicates
dim_location = dim_location.drop_duplicates()

# Reset index
dim_location.reset_index(drop=True, inplace=True)

# Set location_id column using .loc
dim_location.loc[:, 'location_id'] = dim_location.index + 1

# Rearrange columns
dim_location = dim_location[['location_id', 'borough', 'latitude', 'longitude', 'zip_code', 'on_street_name']]

# Print dimension table
print(dim_location)


        location_id    borough   latitude  longitude zip_code   
0                 1   BROOKLYN  40.683304 -73.917274  11233.0  \
1                 2  MANHATTAN  40.751440 -73.973970  10017.0   
2                 3     QUEENS  40.675884 -73.755770  11413.0   
3                 4     QUEENS  40.666840 -73.789410  11434.0   
4                 5      BRONX  40.872620 -73.904686  10463.0   
...             ...        ...        ...        ...      ...   
212251       212252   BROOKLYN  40.640694 -73.996980  11219.0   
212252       212253  MANHATTAN  40.714310 -73.987175  10002.0   
212253       212254      BRONX  40.834340 -73.896970  10456.0   
212254       212255   BROOKLYN  40.594276 -73.978110  11223.0   
212255       212256   BROOKLYN  40.699276 -73.988594  11201.0   

               on_street_name  
0             SARATOGA AVENUE  
1                    3 AVENUE  
2       SPRINGFIELD BOULEVARD  
3        NORTH CONDUIT AVENUE  
4       WEST KINGSBRIDGE ROAD  
...                       .

In [114]:
# Create Date Dimension

# Define a function to calculate the week of the month
def week_of_month(dt):
    year = dt.year
    month = dt.month
    day = dt.day

    cal = calendar.monthcalendar(year, month)
    week_number = (day - 1) // 7 + 1
    return week_number

# Extract relevant date-related columns from your dataset
start_date = car_crash_df['crash_date'].min()
end_date = car_crash_df['crash_date'].max()

# Create a DataFrame for the date dimension
date_dimension = pd.DataFrame({'date': pd.date_range(start_date, end_date, freq='H')})

# Extract attributes
date_dimension['year_number'] = date_dimension['date'].dt.year
date_dimension['quarter_number'] = date_dimension['date'].dt.quarter
date_dimension['month_number'] = date_dimension['date'].dt.month
date_dimension['month_name'] = date_dimension['date'].dt.strftime('%B')
date_dimension['day_number'] = date_dimension['date'].dt.day
date_dimension['day_name'] = date_dimension['date'].dt.strftime('%A')
date_dimension['hour_number'] = date_dimension['date'].dt.hour
date_dimension['date_iso_format'] = date_dimension['date'].apply(lambda x: x.isoformat())
date_dimension['date_id'] = date_dimension['date'].dt.strftime('%Y%m%d%H')

# Define the function to convert crash_time to military time
def convert_to_military_format(time_str):
    return time_str

# Add 'time' column using 'crash_time' from the original dataset
date_dimension['time_militaryformat'] = car_crash_df['crash_time'].apply(convert_to_military_format)


# Add week of the month and week of the year
date_dimension['week_of_month'] = date_dimension['date'].apply(week_of_month)
date_dimension['week_of_year'] = date_dimension['date'].dt.strftime('%U')

# Add crash_time column
#date_dimension['time'] = car_crash_df['crash_time']

# Define the order of columns in the date dimension
new_order = ['date_id', 'date_iso_format','time_militaryformat', 'year_number','quarter_number','month_number','month_name','day_number','day_name', 'hour_number','week_of_month','week_of_year']
date_dimension = date_dimension[new_order]

# Print the date dimension
print(date_dimension.head(25))

       date_id      date_iso_format time_militaryformat  year_number   
0   2012070100  2012-07-01T00:00:00                 NaN         2012  \
1   2012070101  2012-07-01T01:00:00                 NaN         2012   
2   2012070102  2012-07-01T02:00:00                 NaN         2012   
3   2012070103  2012-07-01T03:00:00                 NaN         2012   
4   2012070104  2012-07-01T04:00:00                8:13         2012   
5   2012070105  2012-07-01T05:00:00                 NaN         2012   
6   2012070106  2012-07-01T06:00:00                 NaN         2012   
7   2012070107  2012-07-01T07:00:00                 NaN         2012   
8   2012070108  2012-07-01T08:00:00                 NaN         2012   
9   2012070109  2012-07-01T09:00:00               14:58         2012   
10  2012070110  2012-07-01T10:00:00                 NaN         2012   
11  2012070111  2012-07-01T11:00:00               16:50         2012   
12  2012070112  2012-07-01T12:00:00                 NaN         

In [115]:
print(car_crash_df.columns)

Index(['crash_date', 'crash_time', 'on_street_name',
       'number_of_persons_injured', 'number_of_persons_killed',
       'number_of_pedestrians_injured', 'number_of_pedestrians_killed',
       'number_of_cyclist_injured', 'number_of_cyclist_killed',
       'number_of_motorist_injured', 'number_of_motorist_killed',
       'contributing_factor_vehicle', 'collision_id', 'vehicle_type_code',
       'borough', 'zip_code', 'latitude', 'longitude'],
      dtype='object')


In [116]:
# Create 'facts' table 

# Rename 'collision_id' to 'fact_id'
car_crash_df = car_crash_df.rename(columns={'collision_id': 'fact_id'})

# Step 1: Create contributingfactor_id column
# Creating a dictionary mapping contributing_factor_vehicle to contributingfactor_id
contributingfactor_id_mapping = dict(zip(dim_contributing_factors['contributing_factor'], dim_contributing_factors['contributingfactor_id']))
# Adding contributingfactor_id column to the original dataset using the mapping
car_crash_df['contributingfactor_id'] = car_crash_df['contributing_factor_vehicle'].map(contributingfactor_id_mapping)

# Step 2: Create location_id column
# Creating a dictionary mapping location attributes to location_id
location_attributes = ['borough', 'latitude', 'longitude', 'zip_code', 'on_street_name']
location_id_mapping = dim_location.set_index(location_attributes)['location_id'].to_dict()
# Creating a tuple of location attributes
car_crash_df['location_tuple'] = car_crash_df[location_attributes].apply(tuple, axis=1)
# Adding location_id column to the original dataset using the mapping
car_crash_df['location_id'] = car_crash_df['location_tuple'].map(location_id_mapping)

# Step 3: Create date_id column
car_crash_df['date_id'] = pd.to_datetime(df['crash_date']).dt.strftime('%Y%m%d%H')

# Step 4: Create vehicle_id column
# Creating a dictionary mapping vehicle_type_code to vehicle_id
vehicle_id_mapping = dict(zip(dim_vehicle_type['vehicle_type_code'], dim_vehicle_type['vehicle_id']))
# Adding vehicle_id column to the original dataset using the mapping
car_crash_df['vehicle_id'] = car_crash_df['vehicle_type_code'].map(vehicle_id_mapping)

# Step 5: Reorder columns
new_order = ['fact_id', 'number_of_persons_injured', 'number_of_persons_killed', 'number_of_pedestrians_injured', 
             'number_of_pedestrians_killed', 'number_of_cyclist_injured', 'number_of_cyclist_killed', 
             'number_of_motorist_injured', 'number_of_motorist_killed', 
             'location_id', 'date_id', 'vehicle_id', 'contributingfactor_id']

car_crash_df = car_crash_df[new_order]

# Display the updated DataFrame
car_crash_df.head()

,fact_id,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,location_id,date_id,vehicle_id,contributingfactor_id
4,4486609,0,0,0,0,0,0,0,0,1,2021121400,1,NaN
9,4486519,0,0,0,0,0,0,0,0,2,2021121400,2,1.0
11,4487127,0,0,0,0,0,0,0,0,3,2021121400,2,2.0
14,4486635,2,0,0,0,0,0,2,0,4,2021121400,2,3.0
18,4487040,1,0,0,0,0,0,1,0,5,2021121100,3,4.0


In [117]:
print(car_crash_df.columns)

Index(['fact_id', 'number_of_persons_injured', 'number_of_persons_killed',
       'number_of_pedestrians_injured', 'number_of_pedestrians_killed',
       'number_of_cyclist_injured', 'number_of_cyclist_killed',
       'number_of_motorist_injured', 'number_of_motorist_killed',
       'location_id', 'date_id', 'vehicle_id', 'contributingfactor_id'],
      dtype='object')


In [129]:
# Connection to PostgreSQL DW and create tables code

def load_config(config_file):
    with open(config_file, 'r') as f:
        config = json.load(f)
    return config

def test_connection(config):
    """ function to test connection """
    try:
        connection = psycopg2.connect(**config)
        print("connection established")
        return connection
    except psycopg2.Error as e:
        print(f"Error establishing connection: {e}")
        return None


def test_sql_code(cursor):
    """ function that will test to create a table in PostgreSQL (sql) """
    
    create_schema = """CREATE SCHEMA IF NOT EXISTS "carcrash";"""
    
    create_table_contribfactors = """
        CREATE  TABLE "carcrash".dim_contributingfactors ( 
            contributingfactor_id BIGINT  NOT NULL  ,
            contributing_factor  VARCHAR(255)    ,
            CONSTRAINT pk_dim_contributingfactors PRIMARY KEY ( contributingfactor_id )
    );"""


    create_table_date = """
        CREATE TABLE "carcrash".dim_date ( 
            date_id              BIGINT  NOT NULL  ,
            date_iso_format      DATE    ,
            year_number          INT    ,
            quarter_number       INT    ,
            month_number         INT    ,
            month_name           VARCHAR(50)    ,
            day_number           INT    ,
            day_name             VARCHAR(50)    ,
            hour_number          INT    ,
            week_of_month        INT    ,
            week_of_year         INT    ,
            time_militaryformat  TIME,
            CONSTRAINT pk_dim_date PRIMARY KEY ( date_id )
    );"""

    create_table_location = """
        CREATE  TABLE "carcrash".dim_location ( 
            location_id          BIGINT  NOT NULL  ,
            borough              VARCHAR(255)    ,
            latitude             FLOAT   ,
            longitude            FLOAT   ,
            zip_code             FLOAT    ,
            on_street_name       VARCHAR(255)    ,
            off_street_name      VARCHAR(255)    ,
            CONSTRAINT pk_dim_location PRIMARY KEY ( location_id )
    );"""

    create_table_vehcile_type = """
        CREATE  TABLE "carcrash".dim_vehicle_type ( 
            vehicle_id           BIGINT  NOT NULL  ,
            vehicle_type_code    VARCHAR(255)    ,
            CONSTRAINT pk_dim_vehicle_type PRIMARY KEY ( vehicle_id )
        );""" 


    create_table_facts = """
        CREATE  TABLE "carcrash".facts_crashes ( 
            fact_id BIGINT  NOT NULL  ,
            number_of_persons_injured INT,
            number_of_persons_killed INT,
            number_of_pedestrians_injured INT    ,
            number_of_pedestrians_killed INT    ,
            number_of_cyclist_injured INT    ,
            number_of_cyclist_killed INT    ,
            number_of_motorist_injured INT    ,
            number_of_motorist_killed INT    ,
            location_id INT,
            date_id BIGINT,
            vehicle_id INT,
            contributingfactor_id INT,
            CONSTRAINT pk_facts_crashes PRIMARY KEY(fact_id)
        );"""
    
    #test_pull = """
        #SELECT * FROM carcrash.dim_contributingfactors"""

    
    try:
        cursor.execute(create_table_date)
        print("created date table")
        cursor.execute(create_table_location)
        print("created location table")
        cursor.execute(create_table_vehcile_type)
        print("created table vehicle type")
        cursor.execute(create_table_facts)
        print("created facts table")
    except psycopg2.Error as e:
        print(f"error with code: {e}")


    
# Load config from file
config = load_config('config_dw.json')

# Establish connection using loaded config
connection = test_connection(config)

if connection:
    with connection:
        with connection.cursor() as cursor:
            # insert function that has the SQL code to test
            test_sql_code(cursor)


connection established
error with code: relation "dim_date" already exists



In [130]:
# Database connection URL
def load_config(config_file):
    with open(config_file, 'r') as f:
        config = json.load(f)
    return config

# Load config from file
config = load_config('config_dw.json')

# Create SQLAlchemy engine from config
engine = create_engine(f"postgresql://{config['user']}:{config['password']}@{config['host']}:{config['port']}/{config['database']}")



In [120]:
dim_contributing_factors.to_sql('dim_contributingfactors', schema='carcrash',con=engine, if_exists='append', index = False)

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "pk_dim_contributingfactors"
DETAIL:  Key (contributingfactor_id)=(1) already exists.

[SQL: INSERT INTO carcrash.dim_contributingfactors (contributingfactor_id, contributing_factor) VALUES (%(contributingfactor_id)s, %(contributing_factor)s)]
[parameters: ({'contributingfactor_id': 1, 'contributing_factor': 'Passing Too Closely'}, {'contributingfactor_id': 2, 'contributing_factor': 'Turning Improperly'}, {'contributingfactor_id': 3, 'contributing_factor': 'Reaction to Uninvolved Vehicle'}, {'contributingfactor_id': 4, 'contributing_factor': 'Unspecified'}, {'contributingfactor_id': 5, 'contributing_factor': 'Passing or Lane Usage Improper'}, {'contributingfactor_id': 6, 'contributing_factor': 'Alcohol Involvement'}, {'contributingfactor_id': 7, 'contributing_factor': 'Driver Inattention/Distraction'}, {'contributingfactor_id': 8, 'contributing_factor': 'View Obstructed/Limited'}  ... displ

In [121]:
# Check the data types of all columns in the dim_location df
data_types = dim_location.dtypes
print("Data types of columns:")
print(data_types)

Data types of columns:
location_id         int64
borough            object
latitude          float64
longitude         float64
zip_code           object
on_street_name     object
dtype: object


In [122]:
# Convert the 'zip_code' column to string type to handle NaN values and mixed types
dim_location['zip_code'] = dim_location['zip_code'].astype(str)

# Remove leading and trailing whitespace characters
dim_location['zip_code'] = dim_location['zip_code'].str.strip()

# Remove decimal point and convert to integer, ignoring NaN values
dim_location['zip_code'] = dim_location['zip_code'].str.split('.').str[0]

# Convert to integer, ignoring errors (NaN values will remain as they are)
dim_location['zip_code'] = pd.to_numeric(dim_location['zip_code'], errors='coerce', downcast='integer')

# Print data types to verify the change
print("Data types of columns:")
print(dim_location.dtypes)


Data types of columns:
location_id         int64
borough            object
latitude          float64
longitude         float64
zip_code          float64
on_street_name     object
dtype: object


In [ ]:
dim_location.to_sql('dim_location', schema='carcrash',con=engine, if_exists='append', index = False)

In [ ]:
dim_vehicle_type.to_sql('dim_vehicle_type', schema='carcrash',con=engine, if_exists='append', index = False)

In [124]:
# Convert data types in the date_dimension df to match the table schema
date_dimension['date_id'] = date_dimension['date_id'].astype(int)
date_dimension['date_iso_format'] = pd.to_datetime(date_dimension['date_iso_format'])
date_dimension['week_of_year'] = date_dimension['week_of_year'].astype(int)

print(date_dimension.dtypes)

date_id                         int64
date_iso_format        datetime64[ns]
time_militaryformat            object
year_number                     int32
quarter_number                  int32
month_number                    int32
month_name                     object
day_number                      int32
day_name                       object
hour_number                     int32
week_of_month                   int64
week_of_year                    int64
dtype: object


In [123]:
date_dimension.to_sql('dim_date', schema='carcrash',con=engine, if_exists='append', index = False)

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "pk_dim_date"
DETAIL:  Key (date_id)=(2012070100) already exists.

[SQL: INSERT INTO carcrash.dim_date (date_id, date_iso_format, time_militaryformat, year_number, quarter_number, month_number, month_name, day_number, day_name, hour_number, week_of_month, week_of_year) VALUES (%(date_id)s, %(date_iso_format)s, %(time_militaryformat)s, %(year_number)s, %(quarter_number)s, %(month_number)s, %(month_name)s, %(day_number)s, %(day_name)s, %(hour_number)s, %(week_of_month)s, %(week_of_year)s)]
[parameters: ({'date_id': '2012070100', 'date_iso_format': '2012-07-01T00:00:00', 'time_militaryformat': None, 'year_number': 2012, 'quarter_number': 3, 'month_number': 7, 'month_name': 'July', 'day_number': 1, 'day_name': 'Sunday', 'hour_number': 0, 'week_of_month': 1, 'week_of_year': '27'}, {'date_id': '2012070101', 'date_iso_format': '2012-07-01T01:00:00', 'time_militaryformat': None, 'year_number': 2012

In [125]:
print(car_crash_df.columns)

Index(['fact_id', 'number_of_persons_injured', 'number_of_persons_killed',
       'number_of_pedestrians_injured', 'number_of_pedestrians_killed',
       'number_of_cyclist_injured', 'number_of_cyclist_killed',
       'number_of_motorist_injured', 'number_of_motorist_killed',
       'location_id', 'date_id', 'vehicle_id', 'contributingfactor_id'],
      dtype='object')


In [126]:
# Check duplicates from the facts df
duplicate_rows = car_crash_df[car_crash_df.duplicated(subset=['fact_id'], keep=False)]
print(duplicate_rows)

         fact_id  number_of_persons_injured  number_of_persons_killed   
3999     4487601                          0                         0  \
4000     4487601                          0                         0   
17986    4460366                          0                         0   
17988    4461405                          1                         0   
17991    4514854                          0                         0   
...          ...                        ...                       ...   
2077642  4714342                          1                         0   
2077643  4714047                          0                         0   
2077644  4713861                          0                         0   
2077645  4714310                          1                         0   
2077647  4714349                          0                         0   

         number_of_pedestrians_injured  number_of_pedestrians_killed   
3999                                 0             

In [127]:
# Drop duplicated values
# Specify a subset of columns to consider for identifying duplicates - all columns except 'fact_id' for comparison
subset_cols = ['number_of_persons_injured', 'number_of_persons_killed', 'number_of_pedestrians_injured', 'number_of_pedestrians_killed', 'number_of_cyclist_injured', 'number_of_cyclist_killed', 'number_of_motorist_injured', 'number_of_motorist_killed', 'location_id', 'date_id', 'vehicle_id', 'contributingfactor_id']

# Count the number of non-null values in each row for the subset of columns
non_null_counts = car_crash_df[subset_cols].count(axis=1)

# Add a new column to the DataFrame containing the count of non-null values
car_crash_df['non_null_counts'] = non_null_counts

# Sort the DataFrame by 'non_null_counts' column in ascending order
car_crash_df.sort_values(by='non_null_counts', ascending=True, inplace=True)

# Drop duplicates, keeping the row with the most non-null values
car_crash_df.drop_duplicates(subset='fact_id', keep='last', inplace=True)

# Drop the 'non_null_counts' column as it's no longer needed
car_crash_df.drop(columns=['non_null_counts'], inplace=True)

In [128]:
car_crash_df.to_sql('facts_crashes', schema='carcrash',con=engine, if_exists='append', index = False)

973